In [2]:
import MDAnalysis as mda
import numpy as np
import os
import shutil
import csv
from utils import *
import warnings 

# Suppress warnings specific to MDAnalysis
warnings.filterwarnings("ignore", category=UserWarning, module="MDAnalysis")

# dictionary of charged protein residues with key being the Amber resname and the value being the net charge of that residue
residue_charge_dict = {'ARG':1,'LYS':1,'ASP':-1,'GLU':-1,'MG':2}

In [ ]:
data_dir = 'temp_structures/'
curr_dir = '6'

if curr_dir in ['4','6','11','16']:
    residue_charge_dict['INI'] = -5
    MM_charge = -12
else:
    residue_charge_dict['INI'] = -4
    MM_charge = -11

# read in the system that you are about to model
complex = mda.Universe(data_dir + 'int1/' + curr_dir + '/QM/opt.pdb')
ini = complex.select_atoms("resname INI")
receptor = complex.select_atoms("not resname INI")
C1_ini_index, C2_ini_index, C3_ini_index, O1_ini_index, O2_ini_index, O3_ini_index = get_ini_indexes(ini)
# get the index of these atoms in the context of the complex universe
C1_index = ini.atoms[C1_ini_index].index
C2_index = ini.atoms[C2_ini_index].index
C3_index = ini.atoms[C3_ini_index].index
O1_index = ini.atoms[O1_ini_index].index
O2_index = ini.atoms[O2_ini_index].index
O3_index = ini.atoms[O3_ini_index].index
# guess the position of the hydrogen atom that we want to add
C1_coords = complex.atoms[C1_index].position
C2_coords = complex.atoms[C2_index].position
O1_coords = complex.atoms[O1_index].position
vector_C1_to_C2 = C2_coords - C1_coords
unit_vector = vector_C1_to_C2 / np.linalg.norm(vector_C1_to_C2)
guess_H1_coords = O1_coords + unit_vector * bond_dists['O-H'] # C2 should be located 1.54 A away from C1

# creat a new universe for H
H_u = mda.Universe.empty(1,
                         n_residues=1,
                         trajectory=True) # necessary for adding coordinates
H_count = np.count_nonzero(ini.atoms.types == 'H')

H_u.add_TopologyAttr('name',['H'+str(H_count)])
H_u.add_TopologyAttr('type', ['H'])
H_u.add_TopologyAttr('resname', ['INI'])
H_u.add_TopologyAttr('resid', [1114])
H_u.atoms.positions = [guess_H1_coords]

indices_to_remove = [C3_index,O2_index,O3_index]
# Select atoms to keep (all atoms excluding the ones in indices_to_remove)
mask = ~np.isin(ini.atoms.indices, indices_to_remove)
atoms_to_keep = ini.atoms[mask]
modified_ini_universe = Merge(atoms_to_keep)
# write a pdb for just the INO molecule
ino_output_dir = data_dir + 'int2/' + curr_dir + '/amber_prep/' 
ino_universe = mda.Merge(modified_ini_universe.atoms,H_u.atoms)
for atom in ino_universe.atoms:
    atom.residue.resid = 1
    atom.residue.resname = "INO"

write_universe(ino_output_dir,'ino.pdb',ino_universe)

# add the substrate and ThDP to a single universe
ino_complex = mda.Merge(ino_universe.atoms,receptor.atoms)

write_universe(ino_output_dir,'ino_complex.pdb',ino_complex)
edit_protein_files(ino_output_dir,'ino_complex.pdb')
    
    